In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydicom
!pip install opendatasets

In [ ]:
# first, load the libraries
import os
import random

import numpy as np
import pandas as pd
import torch
import csv
from copy import deepcopy
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from skimage.io import imread, imsave
from skimage.exposure import rescale_intensity
from skimage.filters import gaussian
from skimage.transform import rotate
from tqdm import tqdm
from torchvision import transforms
from torchvision.models import densenet121
import matplotlib.pyplot as plt

import opendatasets as od

In [ ]:
# download the dataset form kaggle ( the data set is pre processed and is private)
# my username and APi key is needed

od.download(
    "https://www.kaggle.com/datasets/barbaracharles/reduced-bc-duke")

In [ ]:
class DBCDataset_train(Dataset):
    def __init__(self, data_dir, img_size):
        self.data_dir = data_dir
        self.img_size = img_size

        # assign labels to data within this Dataset
        self.labels = None
        self.create_labels()

    def create_labels(self):
        # create and store a label (positive/1 or negative/0 for each image)
        # each label is the tuple: (img filename, label number (0 or 1))
        labels = []
        print('building DBC dataset labels.')
        # iterate over each class
        for target, target_label in enumerate(['neg', 'pos']):
            case_dir = os.path.join(self.data_dir, target_label)
            #counter = 0 # Uncomment to use Dataset E
            for fname in os.listdir(case_dir):
                if '.png' in fname:
                    fpath = os.path.join(case_dir, fname)
                    #if counter % 2 == 0: # Uncomment to use Dataset E
                    labels.append((fpath, target)) # Add a tab to use Dataset E
                    #counter += 1 # Uncomment to use Dataset E

        self.labels = labels

    def normalize(self, img):
        # normalize image pixel values to range [0, 255]
        # img expected to be array

        # convert uint16 -> float
        img = img.astype(np.float32) * 255. / img.max()
        # convert float -> unit8
        img = img.astype(np.uint8)

        return img

    def __getitem__(self, idx):
        # required method for accessing data samples
        # returns data with its label
        fpath, target = self.labels[idx]

        # load img from file (png or jpg)
        img_arr = imread(fpath, as_gray=True)

        # Apply Gaussian blur randomly to every other image
        if idx % 6 == 1:  # Apply Gaussian blur on the second of every 4 images
            sigma = random.uniform(0.5, 3.0)  # Randomly select sigma for Gaussian blur
            img_arr = gaussian(img_arr, sigma=sigma)
            img_arr = (img_arr * 255).astype(np.uint8)

        # Increase brightness on the third of every 4 images
        elif idx % 6 == 2:
            brightness_increase = 0.3
            img_arr = rescale_intensity(img_arr, in_range='image', out_range=(0, 1))  # Rescale intensity to range [0, 1]
            img_arr += brightness_increase
            img_arr = np.clip(img_arr, 0, 1)  # Clip values to ensure they remain within [0, 1] range
            img_arr = (img_arr * 255).astype(np.uint8)

        # Rotate 25 degrees on the fourth of every 4 images
        elif idx % 6 == 3:
            img_arr = rotate(img_arr, angle=25)
            img_arr = (img_arr * 255).astype(np.uint8)


        # normalize image
        img_arr = img_arr / 255.0

        # convert to tensor (PyTorch matrix)
        data = torch.from_numpy(img_arr)
        data = data.type(torch.FloatTensor)

        # add image channel dimension (to work with neural network)
        data = torch.unsqueeze(data, 0)

        # resize image
        data = transforms.Resize((self.img_size, self.img_size))(data)

        return data, target

    def __len__(self):
        # required method for getting size of dataset
        return len(self.labels)



class DBCDataset(Dataset):
    def __init__(self, data_dir, img_size):
        self.data_dir = data_dir
        self.img_size = img_size

        # assign labels to data within this Dataset
        self.labels = None
        self.create_labels()

    def create_labels(self):
        # create and store a label (positive/1 or negative/0 for each image)
        # each label is the tuple: (img filename, label number (0 or 1))
        labels = []
        print('building DBC dataset labels.')
        # iterate over each class
        for target, target_label in enumerate(['neg', 'pos']):
            case_dir = os.path.join(self.data_dir, target_label)
            for fname in os.listdir(case_dir):
                if '.png' in fname:
                    fpath = os.path.join(case_dir, fname)
                    labels.append((fpath, target))

        self.labels = labels

    def normalize(self, img):
        # normalize image pixel values to range [0, 255]
        # img expected to be array

        # convert uint16 -> float
        img = img.astype(np.float32) * 255. / img.max()
        # convert float -> unit8
        img = img.astype(np.uint8)

        return img

    def __getitem__(self, idx):
        # required method for accessing data samples
        # returns data with its label
        fpath, target = self.labels[idx]

        # load img from file (png or jpg)
        img_arr = imread(fpath, as_gray=True)

        # normalize image
        #img_arr = self.normalize(img_arr)

        img_arr = img_arr / 255.0

        # convert to tensor (PyTorch matrix)
        data = torch.from_numpy(img_arr)
        data = data.type(torch.FloatTensor)

        # add image channel dimension (to work with neural network)
        data = torch.unsqueeze(data, 0)

        # resize image
        data = transforms.Resize((self.img_size, self.img_size))(data)

        return data, target

    def __len__(self):
        # required method for getting size of dataset
        return len(self.labels)

In [ ]:
# Create the dataset

img_size=128

train_data_dir='/content/reduced-bc-duke/reduced_dataset/png_out/training'
val_data_dir='/content/reduced-bc-duke/reduced_dataset/png_out/validation'
test_data_dir='/content/reduced-bc-duke/reduced_dataset/png_out/testing'

training_dataset = DBCDataset_train(train_data_dir,img_size)
validation_dataset = DBCDataset(val_data_dir,img_size)
testing_dataset = DBCDataset(test_data_dir,img_size)

print(f"length of the training dataset: {len(training_dataset)}")
print(f"length of the validation dataset: {len(validation_dataset)}")
print(f"length of the testing dataset: {len(testing_dataset)}")

In [ ]:
# GPUs # to run only online if a gpu is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on {}'.format(device))

In [ ]:
# Set the batch sizes and load the dataset

train_batchsize = 32
eval_batchsize = 8
train_loader = DataLoader(training_dataset,
                          batch_size=train_batchsize,
                          shuffle=True
                          # images are loaded in random order
                          )

validation_loader = DataLoader(validation_dataset,
                               batch_size=eval_batchsize,
                               shuffle=True)

test_loader = DataLoader(testing_dataset,
                         batch_size=eval_batchsize)



In [ ]:
# set random seeds for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
#Load the model and set the hyperparameters

model = densenet121(pretrained=False)

# Change the input size of the architecture
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

model = model.to(device)


criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
weight_decay=0.0001
#error_minimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9, weight_decay=weight_decay)
error_minimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# Set up the learning rate scheduler
lr_scheduler = ReduceLROnPlateau(error_minimizer, mode='min', patience=1, factor=0.5, min_lr=1e-6)


epochs = 20





In [ ]:
# Create log files to be able to rereive trainig information later

!mkdir -p "/content/drive/MyDrive/Densenet"
model_name = f"Densenet_{train_batchsize}_{eval_batchsize}_{epochs}_{learning_rate}_{weight_decay}_{img_size}_Adam"

filename = f'{model_name}.txt'
log_epoch = f'{model_name}_epoch.txt'
dir_path = f'/content/drive/MyDrive/Densenet/{model_name}'
!mkdir -p $dir_path
log_file_path = os.path.join(dir_path, filename)
epoch_log_file_path = os.path.join(dir_path, log_epoch)

with open(log_file_path, "w") as f:
  f.write(f'Model : Densenet, number of epochs: {epochs}, training batch size: {train_batchsize}, eval batch size: {eval_batchsize}, error minimiser learning rate: {learning_rate}\n')



In [ ]:
# Training and validation of the model

model_final = deepcopy(model)

best_val_accuracy = 0.0
best_val_loss = float('inf')  # Initialize with infinity
epochs_without_improvement = 0
max_epochs_without_improvement = 2
# used to pick the best-performing model on the validation set

# for training visualization later
train_accuracy = []
val_accuracy = []

# for storing loss function values
train_loss = []  # list to store training losses
val_loss = []    # list to store validation losses

# for storing specificity and sensitivity
train_specificity = []
train_sensitivity = []
val_specificity = []
val_sensitivity = []

# training loop
for epoch in range(epochs):
    # set network to training mode, so that its parameters can be changed
    model.train()

    # print training info
    print(f"### Epoch {epoch}:")
    with open(log_file_path, "a") as f:
      f.write(f"### Epoch {epoch}:\n")
    with open(epoch_log_file_path, "a") as f:
      f.write(f"### Epoch {epoch}:\n")

    # statistics needed to compute classification accuracy:
    # the total number of image examples trained on
    total_train_examples = 0

    # the number of examples classified correctly
    num_correct_train = 0

    # variable to store training loss in this epoch
    epoch_train_loss = 0.0

    # variables for specificity and sensitivity
    true_positives_train = 0
    true_negatives_train = 0
    false_positives_train = 0
    false_negatives_train = 0

    # iterate over the training set once
    for batch_index, (inputs, targets) in tqdm(enumerate(train_loader),
                                               total=len(training_dataset) // train_batchsize):
        # load the data onto the computation device.
        # inputs are a tensor of shape:
        #   (batch size, number of channels, image height, image width).
        # targets are a tensor of one-hot-encoded class labels for the inputs,
        #   of shape (batch size, number of classes)
        # in other words,
        inputs = inputs.to(device)
        targets = targets.to(device)

        # reset changes (gradients) to parameters
        error_minimizer.zero_grad()

        # get the network's predictions on the training set batch
        predictions = model(inputs)

        # evaluate the error, and estimate
        #   how much to change the network parameters
        loss = criterion(predictions, targets)
        loss.backward()

        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Uncomment to use gradient clipping and adjust the norm is needed


        # change parameters
        error_minimizer.step()

        # accumulate loss for this epoch
        epoch_train_loss += loss.item()

        # calculate predicted class label
        # the .max() method returns the maximum entries, and their indices;
        # we just need the index with the highest probability,
        #   not the probability itself.
        _, predicted_class = predictions.max(1)
        total_train_examples += predicted_class.size(0)

        # calculate true positives, true negatives, false positives, and false negatives
        true_positives_train += ((predicted_class == 1) & (targets == 1)).sum().item()
        true_negatives_train += ((predicted_class == 0) & (targets == 0)).sum().item()
        false_positives_train += ((predicted_class == 1) & (targets == 0)).sum().item()
        false_negatives_train += ((predicted_class == 0) & (targets == 1)).sum().item()

        with open(epoch_log_file_path, "a") as f:
          f.write(f"Batch {batch_index + 1}/{len(train_loader)} - Loss: {loss.item()}\n")

    # calculate specificity and sensitivity
    epoch_train_specificity = true_negatives_train / (true_negatives_train + false_positives_train)
    epoch_train_sensitivity = true_positives_train / (true_positives_train + false_negatives_train)
    # store specificity and sensitivity
    train_specificity.append(epoch_train_specificity)
    train_sensitivity.append(epoch_train_sensitivity)


    # get results
    # total prediction accuracy of network on training set
    epoch_train_accuracy = (true_positives_train+true_negatives_train)/total_train_examples
    print(f"Training accuracy: {epoch_train_accuracy}")
    with open(log_file_path, "a") as f:
      f.write(f"Training accuracy: {epoch_train_accuracy}\n")
    train_accuracy.append(epoch_train_accuracy)

    # average training loss for this epoch
    epoch_train_loss /= len(train_loader)
    train_loss.append(epoch_train_loss)

    # predict on validation set (similar to training set):
    total_val_examples = 0
    num_correct_val = 0

    # variable to store validation loss in this epoch
    epoch_val_loss = 0.0

    # variables for specificity and sensitivity
    true_positives_val = 0
    true_negatives_val = 0
    false_positives_val = 0
    false_negatives_val = 0

    # switch network from training mode (parameters can be trained)
    #   to evaluation mode (parameters can't be trained)
    model.eval()

    with torch.no_grad():  # don't save parameter changes
        #                      since this is not for training
        for batch_index, (inputs, targets) in tqdm(enumerate(validation_loader),
                                                   total=len(validation_dataset) // eval_batchsize):
            inputs = inputs.to(device)
            targets = targets.to(device)
            predictions = model(inputs)

            _, predicted_class = predictions.max(1)
            total_val_examples += predicted_class.size(0)
            num_correct_val += predicted_class.eq(targets).sum().item()

            # calculate validation loss
            loss = criterion(predictions, targets)
            epoch_val_loss += loss.item()

            # calculate true positives, true negatives, false positives, and false negatives
            true_positives_val += ((predicted_class == 1) & (targets == 1)).sum().item()
            true_negatives_val += ((predicted_class == 0) & (targets == 0)).sum().item()
            false_positives_val += ((predicted_class == 1) & (targets == 0)).sum().item()
            false_negatives_val += ((predicted_class == 0) & (targets == 1)).sum().item()

    # calculate specificity and sensitivity
    epoch_val_specificity = true_negatives_val / (true_negatives_val + false_positives_val)
    epoch_val_sensitivity = true_positives_val / (true_positives_val + false_negatives_val)
    # store specificity and sensitivity
    val_specificity.append(epoch_val_specificity)
    val_sensitivity.append(epoch_val_sensitivity)

    # get results
    # total prediction accuracy of network on validation set
    epoch_val_accuracy = (true_positives_val + true_negatives_val) / total_val_examples
    print(f"Validation accuracy: {epoch_val_accuracy}")
    val_accuracy.append(epoch_val_accuracy)
    with open(log_file_path, "a") as f:
      f.write(f"Validation accuracy: {epoch_val_accuracy}\n")

    # average validation loss for this epoch
    epoch_val_loss /= len(validation_loader)
    val_loss.append(epoch_val_loss)

    # Finally, save model if the validation accuracy is the best so far
    if epoch_val_accuracy > best_val_accuracy:
        best_val_accuracy = epoch_val_accuracy
        best_train_accuracy = epoch_train_accuracy
        best_train_specificity = epoch_train_specificity
        best_train_sensitivity = epoch_train_sensitivity
        best_val_specificity = epoch_val_specificity
        best_val_sensitivity = epoch_val_sensitivity
        print("Validation accuracy improved; saving model.")
        with open(log_file_path, "a") as f:
          f.write("Validation accuracy improved; saving model.\n")
        model_final = deepcopy(model)

    # Get the current learning rate
    with open(log_file_path, "a") as f:
        f.write(f"Current learning rate: {error_minimizer.param_groups[0]['lr']}\n")
    # Reduce the learnign rate
    val_loss_avg = epoch_val_loss / len(validation_loader)
    lr_scheduler.step(val_loss_avg)

    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
    # If validation loss hasn't improved for 'max_epochs_without_improvement' epochs, stop training
    if epochs_without_improvement >= max_epochs_without_improvement:
        print(f"No improvement in validation loss for {max_epochs_without_improvement} epochs. Stopping early.")
        break





with open(log_file_path, "a") as f:
  f.write(f"Training Accuracies: {train_accuracy}\n")
  f.write(f"Training Specificities: {train_specificity}\n")
  f.write(f"Training Sensitivities: {train_sensitivity}\n")
  f.write(f"Traingin Loss: {train_loss}\n")
  f.write(f"Validation Accuracies: {val_accuracy}\n")
  f.write(f"Validation Specificities: {val_specificity}\n")
  f.write(f"Validation Sensitivities: {val_sensitivity}\n")
  f.write(f"Validation Loss: {val_loss}\n")

In [ ]:
# Save the trained model
model_save_name = f'{model_name}.pt'
path = F"/content/drive/MyDrive/{model_save_name}"
torch.save(model_final.state_dict(), path)

In [ ]:



# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(train_accuracy, label='Training Accuracy', color='blue')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

filename = f'Acc_{model_name}.png'
graph_file_path = os.path.join(dir_path, filename)
plt.savefig(graph_file_path)
plt.show()

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Training Loss', color='blue')
plt.plot(val_loss, label='Validation Loss', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

filename = f'Loss_{model_name}.png'
graph_file_path = os.path.join(dir_path, filename)
plt.savefig(graph_file_path)
plt.show()

# Plot training and validation specificity
plt.figure(figsize=(10, 5))
plt.plot(train_specificity, label='Training Specificity', color='blue')
plt.plot(val_specificity, label='Validation Specificity', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Specificity')
plt.title('Training and Validation Specificity')
plt.legend()

filename = f'Spec_{model_name}.png'
graph_file_path = os.path.join(dir_path, filename)
plt.savefig(graph_file_path)
plt.show()

# Plot training and validation sensitivity
plt.figure(figsize=(10, 5))
plt.plot(train_sensitivity, label='Training Sensitivity', color='blue')
plt.plot(val_sensitivity, label='Validation Sensitivity', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Sensitivity')
plt.title('Training and Validation Sensitivity')
plt.legend()

filename = f'Sens_{model_name}.png'
graph_file_path = os.path.join(dir_path, filename)
plt.savefig(graph_file_path)
plt.show()

In [ ]:
# for data visualization
total_test_examples = 0
num_correct_test = 0

true_positives_test = 0
false_positives_test = 0
true_negatives_test = 0
false_negatives_test = 0

# see how well the final trained model does on the test set
with torch.no_grad():  # don't save parameter gradients/changes since this is not for model training
    for batch_index, (inputs, targets) in enumerate(test_loader):
        # make predictions
        inputs = inputs.to(device)
        targets = targets.to(device)
        predictions = model_final(inputs)

        # compute prediction statistics
        _, predicted_class = predictions.max(1)
        total_test_examples += predicted_class.size(0)
        num_correct_test += predicted_class.eq(targets).sum().item()

        # thanks to
        #   https://gist.github.com/the-bass/cae9f3976866776dea17a5049013258d
        confusion_vector = predicted_class / targets
        # num_true_pos = torch.sum(confusion_vector == 1).item()
        # num_false_pos = torch.sum(confusion_vector == float('inf')).item()

        true_positive = torch.sum(confusion_vector == 1).item()
        false_positive = torch.sum(confusion_vector == float('inf')).item()
        true_negative = torch.sum(torch.isnan(confusion_vector)).item()
        false_negative = torch.sum(confusion_vector == 0).item()

        true_positives_test += true_positive
        false_positives_test += false_positive
        true_negatives_test += true_negative
        false_negatives_test += false_negative


# get total results
# total prediction accuracy of network on test set
test_accuracy = (true_positives_test + true_negatives_test) / total_test_examples
test_specificity = true_negatives_test / (true_negatives_test + false_positives_test + 1e-8)
test_sensitivity = true_positives_test / (true_positives_test + false_negatives_test + 1e-8)
print(f"Test set accuracy: {test_accuracy}\n")
print(f"Test set specificity: {test_specificity}\n")
print(f"Test set sensitivity: {test_sensitivity}\n")
print(f"{true_positives_test} true positive classifications,\n {false_positives_test} false positive classifications\n")
print(f"{true_negatives_test} true negative classification,\n {false_negatives_test}false negative classification\n")

with open(log_file_path, "a") as f:
    f.write(f"Test set accuracy: {test_accuracy}\n")
    f.write(f"Test set specificity: {test_specificity}\n")
    f.write(f"Test set sensitivity: {test_sensitivity}\n")
    f.write(f"{true_positives_test} true positive classifications,\n {false_positives_test} false positive classifications\n")
    f.write(f"{true_negatives_test} true negative classification,\n {false_negatives_test}false negative classification\n")

In [ ]:
#Model	Training batch	Eval batch	epochs Learning rate	test acc	test spec	test sens	train acc	train spec	train sens	val acc	val spec	val sens
data = ['Densenet', train_batchsize, eval_batchsize, epochs, learning_rate, test_accuracy, test_specificity, test_sensitivity, best_train_accuracy, best_train_specificity, best_train_sensitivity,  best_val_accuracy, best_val_specificity, best_val_sensitivity ]


results_file_path = "/content/drive/MyDrive/ResultsClass.csv"

with open(results_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    # Write each row of new data to the CSV file
    writer.writerow(data)